# TensorRT Model Optimized
TensorRT-based applications perform up to 40x faster than CPU-only platforms during inference. With TensorRT, you can optimize neural network models trained in all major frameworks, calibrate for lower precision with high accuracy
See https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.compiler.tensorrt import trt_convert as trt

import os

**ATENCIÓN:** Última capa del modelo debe estar con name="LastLayer".

e.g.:  *keras.layers.Dense(y.shape[1], activation='softmax', **name ="LastLayer"**)*

In [5]:
PATH='data/Final/'
modelo='model_n1n2n3_v10.h5'

### 1) Abrimos el modelo KERAS ".h5"

In [6]:
model_X = tf.keras.models.load_model(PATH+modelo)

In [7]:
#Cargamos un dataset de ejemplo
noneDataset=np.load(PATH+'none.npy')

In [8]:
#Probamos modelo
x1=noneDataset[3]
x1=x1.reshape(1,36)

output=model_X.predict(x1)
clase=np.argmax(output)

print(output)
print(clase)

[[1.8608925e-17 2.4994437e-05 1.4050877e-25 4.5188310e-14 9.9997497e-01]]
4


### 2) Lo guardo en formato TF ".pb"

In [9]:
#SAVE THE MODEL model_name = "Model_trt"
model_path_pb = PATH+'/model_pb' # define path to save model in saved model format

os.makedirs(model_path_pb, exist_ok=True)

tf.saved_model.save(model_X, model_path_pb) 
print("model saved under: ",model_path_pb)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: data/Final//model_pb/assets
model saved under:  data/Final//model_pb


### 3) Optimizamos el modelo con TensorRT

In [10]:
# CONVERT TO TENSOR RT

model_path_trt = PATH+'/model_trt' # define path to save model in saved model format
os.makedirs(model_path_trt, exist_ok=True)

print("\nconverting to trt-model")
converter = trt.TrtGraphConverterV2(input_saved_model_dir=model_path_pb) 
# we feed the saved model into this converter function

print("\nconverter.convert")
converter.convert()

print("\nconverter.save")
converter.save(model_path_trt) # we save the converted model

print("trt-model saved under: ",model_path_trt)


converting to trt-model
INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)

converter.convert

converter.save
INFO:tensorflow:Assets written to: data/Final//model_trt/assets
trt-model saved under:  data/Final//model_trt


### 4) Cargamos modelo optimizado (trt) y probamos.

In [11]:
#LOAD NEW MODEL
loaded = tf.saved_model.load(PATH+'/model_trt')  # loading the converted model
infer = loaded.signatures["serving_default"]

In [12]:
x1=noneDataset[3]
x1=x1.reshape(1,36)

In [13]:
#PREDICTION
output = infer(tf.constant(x1,dtype=float))['LastLayer']   

clase=np.argmax(output)
print(output)
print(clase)

tf.Tensor([[1.8608925e-17 2.4994437e-05 1.4050877e-25 4.5188310e-14 9.9997497e-01]], shape=(1, 5), dtype=float32)
4
